In [1]:
from matplotlib import pyplot as plt
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import progress
from tqdm.autonotebook import tqdm
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import intake
import fsspec
import seaborn as sns
#import gcsfs
import cftime
import pandas as pd
import xesmf as xe
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

/var/folders/k1/5_y16ccj4zv7lcjp767yd1b80000gn/T/ipykernel_1571/2233006168.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
/Users/hrh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from dask.distributed import Client

#client = Client(n_workers=6, memory_limit='2GB')
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50376,Workers: 5
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:50392,Total threads: 2
Dashboard: http://127.0.0.1:50396/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:50379,


In [3]:
chicago_lat=41.88
chicago_lon=(360-87.6298)%360
#
cvals = '/Users/hrh/Desktop/TwoMoments21/cvals/cmip6/'

In [4]:
# calculate global means

def get_lat_name(ds):
    for lat_name in ['lat', 'latitude']:
        if lat_name in ds.coords:
            return lat_name
    raise RuntimeError("Couldn't find a latitude coordinate")

def global_mean(ds):
    lat = ds[get_lat_name(ds)]
    weight = np.cos(np.deg2rad(lat))
    weight /= weight.mean()
    other_dims = set(ds.dims) - {'quantile'}
    return (ds * weight).mean(other_dims)

In [5]:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [6]:
# 2. Search for maximum temperature for son 30
expts = ['ssp370','historical']

cat = col.search(
    experiment_id=expts,
    table_id='day',
    variable_id='tasmax',
    #grid_label='gn'
)

query = dict(
    experiment_id=expts,
    table_id='day',
    variable_id=['tasmax'],
    member_id = 'r1i1p1f1',
)

col_subset = col.search(require_all_on=["source_id"], **query)
col_subset.df.groupby("source_id")[
    ["experiment_id", "variable_id", "table_id","member_id"]
].nunique()

,experiment_id,variable_id,table_id,member_id
source_id,,,,
ACCESS-CM2,2,1,1,1
AWI-CM-1-1-MR,2,1,1,1
BCC-ESM1,2,1,1,1
CMCC-ESM2,2,1,1,1
CanESM5,2,1,1,1
EC-Earth3,2,1,1,1
EC-Earth3-AerChem,2,1,1,1
EC-Earth3-Veg-LR,2,1,1,1
FGOALS-g3,2,1,1,1


In [7]:
df = col_subset.df
model_counts = df.groupby('source_id').size()
print(model_counts)

source_id
ACCESS-CM2           2
AWI-CM-1-1-MR        2
BCC-ESM1             2
CMCC-ESM2            2
CanESM5              2
EC-Earth3            2
EC-Earth3-AerChem    2
EC-Earth3-Veg-LR     2
FGOALS-g3            2
GFDL-ESM4            2
INM-CM4-8            2
INM-CM5-0            2
IPSL-CM6A-LR         2
KACE-1-0-G           2
MIROC6               2
MPI-ESM-1-2-HAM      2
MPI-ESM1-2-HR        2
MPI-ESM1-2-LR        2
MRI-ESM2-0           2
NorESM2-LM           2
NorESM2-MM           2
dtype: int64


In [8]:
def drop_all_bounds(ds):
    drop_vars = [vname for vname in ds.coords
                 if (('_bounds') in vname ) or ('_bnds') in vname]
    return ds.drop(drop_vars)

def open_dset(df):
    assert len(df) == 1
    ds = xr.open_zarr(fsspec.get_mapper(df.zstore.values[0]), consolidated=True)
    return drop_all_bounds(ds)

def open_delayed(df):
    return dask.delayed(open_dset)(df)

from collections import defaultdict
dsets = defaultdict(dict)

for group, df in col_subset.df.groupby(by=['source_id', 'experiment_id']):
    dsets[group[0]][group[1]] = open_delayed(df)

In [9]:
%%time
# Trigger computation
dsets_ = dask.compute(dict(dsets))[0]

/Users/hrh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/hrh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/hrh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/hrh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={n

CPU times: user 2.61 s, sys: 363 ms, total: 2.97 s
Wall time: 13.2 s


In [10]:
#Define coarse grid to regrid on
ds_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 91, 3)),
                     'lon': (['lon'], np.arange(0, 361, 3)),})

In [20]:
def drop_feb29(ds):
    # Check if the dataset's calendar is not '360_day'
    calendar = ds.time.encoding.get('calendar', None)
    print(ds.attrs['source_id'],calendar)
    if calendar != '360_day':
        ds = ds.convert_calendar('365_day')
    return ds


def to_daily(ds):
    # Check and deal with different datetime types
    if isinstance(ds['time'].values[0], np.datetime64):
        pass
    elif isinstance(ds['time'].values[0], cftime.datetime):
        pass
    else:
        # convert time coordinate to datetime64 objects
        ds['time'] = ds['time'].astype('datetime64[ns]')
    year      = ds.time.dt.year
    dayofyear = ds.time.dt.dayofyear

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), dayofyear=("time", dayofyear.data))

    # reshape the array to (..., "day", "year")
    return ds.set_index(time=("year", "dayofyear")).unstack("time")  


def extract_data(ds):
    """
    Extract data for djf from the dataset 'ds' for specific time and spatial range.

    Parameters:
    - ds (xarray.Dataset): Input dataset

    Returns:
    - xarray.Dataset: Dataset subsetted for djf and the specified space and time range.
    """    

    subset1 = ds.sel(year=slice(1850, 1879))
    subset2 = ds.sel(year=slice(2071, 2100))
    
    subset = xr.concat([subset1, subset2], dim='year')  

    return subset

def is_leap(year):
    """Check if a year is a leap year."""
    return (year % 4 == 0) and ((year % 100 != 0) or (year % 400 == 0))



In [21]:
quants = np.linspace(0,1.0,30)


def compute_quantiles(ds, quantiles=quants):
    return ds.chunk(dict(year=-1)).quantile(quantiles, dim='year',skipna=True)

def regrid(ds, ds_out):
    experiment_id = ds.attrs['experiment_id']
    source_id     = ds.attrs['source_id']
    #regrid
    regridder = xe.Regridder(ds, ds_out, 'nearest_s2d', reuse_weights=False)
    ds_new    = regridder(ds)
    
#     #Assign back attributes as regirdder would have deleted attributes 
    ds_new.attrs['experiment_id'] = experiment_id
    ds_new.attrs['source_id'] = source_id
    
    #print(ds_new.attrs['experiment_id'],ds_new.attrs['source_id'])
    #print(ds_new)
    return ds_new

def process_data(ds, quantiles=quants):
    ds = ds.pipe(drop_feb29).pipe(to_daily).pipe(extract_data)
    
    if len(ds['year']) == 0:
        print("The dataset is empty. Skipping...")
        return None
    
    if len(ds['dayofyear'])<365:
        print('The dataset has less than 365 days. Skipping ..')
        return None
    
    # # Remove 'time' coordinate
    # ds = ds.set_index(time=("year", "dayofyear")).unstack("time")  
   
    
    return (ds.pipe(regrid, ds_out=ds_out)
           )


In [22]:
with progress.ProgressBar():

    expt_da = xr.DataArray(expts, dims='experiment_id', name='experiment_id',
                           coords={'experiment_id': expts})

    # Initialize an Empty Dictionary for Aligned Datasets:
    dsets_aligned = {}

    # Iterate Over dsets_ Dictionary:

    for k, v in tqdm(dsets_.items()):
        # Initialize a dictionary for this source_id
        dsets_aligned[k] = {}
        
        skip_source_id = False

        for expt in expts:
            ds = v[expt].pipe(process_data)

            # Check if the dataset is empty and skip this source_id if so
            if ds is None:
                print(f"Skipping {expt} for {k} because the dataset is empty")
                skip_source_id = True
                break
            
            # Store the dataset in the dictionary
            # dsets_aligned[k][expt] = ds
            # Compute the dataset and store it in the dictionary
            dsets_aligned[k][expt] = ds.compute()
            print(dsets_aligned[k][expt])

        if skip_source_id:
            del dsets_aligned[k]
            continue


  0%|          | 0/21 [00:00<?, ?it/s]

ACCESS-CM2 proleptic_gregorian


/Users/hrh/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<xarray.Dataset>
Dimensions:    (year: 30, dayofyear: 365, lat: 61, lon: 121)
Coordinates:
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
    height     float64 2.0
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
Data variables:
    tasmax     (year, dayofyear, lat, lon) float32 258.6 258.6 ... 268.1 268.1
Attributes:
    regrid_method:  nearest_s2d
    experiment_id:  ssp370
    source_id:      ACCESS-CM2
ACCESS-CM2 proleptic_gregorian
<xarray.Dataset>
Dimensions:    (year: 30, dayofyear: 365, lat: 61, lon: 121)
Coordinates:
  * year       (year) int64 1850 1851 1852 1853 1854 ... 1876 1877 1878 1879
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
    height     float64 2.0
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72

/Users/hrh/anaconda3/lib/python3.9/site-packages/xarray/core/dataset.py:4983: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(


...

<xarray.Dataset>
Dimensions:    (year: 30, dayofyear: 365, lat: 61, lon: 121)
Coordinates:
  * year       (year) int64 1850 1851 1852 1853 1854 ... 1876 1877 1878 1879
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
    height     float64 2.0
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
Data variables:
    tasmax     (year, dayofyear, lat, lon) float32 243.2 243.2 ... 264.5 264.5
Attributes:
    regrid_method:  nearest_s2d
    experiment_id:  historical
    source_id:      MRI-ESM2-0
NorESM2-LM noleap
<xarray.Dataset>
Dimensions:    (year: 30, dayofyear: 365, lat: 61, lon: 121)
Coordinates:
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
    height     float64 2.0
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81

/Users/hrh/anaconda3/lib/python3.9/site-packages/xarray/core/dataset.py:4983: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(


<xarray.Dataset>
Dimensions:    (year: 30, dayofyear: 365, lat: 61, lon: 121)
Coordinates:
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
    height     float64 2.0
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
Data variables:
    tasmax     (year, dayofyear, lat, lon) float32 253.9 253.9 ... 255.4 255.4
Attributes:
    regrid_method:  nearest_s2d
    experiment_id:  ssp370
    source_id:      NorESM2-MM
NorESM2-MM noleap


/Users/hrh/anaconda3/lib/python3.9/site-packages/xarray/core/dataset.py:4983: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  result = result._unstack_full_reindex(


<xarray.Dataset>
Dimensions:    (year: 30, dayofyear: 365, lat: 61, lon: 121)
Coordinates:
  * year       (year) int64 1850 1851 1852 1853 1854 ... 1876 1877 1878 1879
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
    height     float64 2.0
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
Data variables:
    tasmax     (year, dayofyear, lat, lon) float32 245.6 245.6 ... 236.1 236.1
Attributes:
    regrid_method:  nearest_s2d
    experiment_id:  historical
    source_id:      NorESM2-MM


In [ ]:
# with progress.ProgressBar():
#     dsets_aligned_ = dask.compute(dsets_aligned)[0]

In [23]:
source_ids = list(dsets_aligned.keys())
source_da = xr.DataArray(source_ids, dims='source_id', name='source_id',
                         coords={'source_id': source_ids})

# final_ds = {expt: xr.concat([ds.get(expt, xr.Dataset()).reset_coords(drop=True)
#                              for ds in dsets_aligned.values()],
#                             dim=source_da)
#             for expt in expts}

final_ds = {expt: xr.concat([ds[expt].reset_coords(drop=True)
                             for ds in dsets_aligned.values() if expt in ds and ds[expt] is not None],
                            dim=source_da, coords='minimal')
            for expt in expts}

final_ds

{'ssp370': <xarray.Dataset>
 Dimensions:    (source_id: 18, year: 30, dayofyear: 365, lat: 61, lon: 121,
                 bnds: 2)
 Coordinates:
   * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
   * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
   * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
   * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
   * bnds       (bnds) float64 1.0 2.0
   * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
 Data variables:
     tasmax     (source_id, year, dayofyear, lat, lon) float32 258.6 ... 255.4
 Attributes:
     regrid_method:  nearest_s2d
     experiment_id:  ssp370
     source_id:      ACCESS-CM2,
 'historical': <xarray.Dataset>
 Dimensions:    (source_id: 18, year: 30, dayofyear: 365, lat: 61, lon: 121,
                 bnds: 2)
 Coordinates:
   * year       (year) int64 1850 1851 1852 1853 1854 ... 1876 1

In [27]:
final_ds_pi = xr.concat([ds['historical'].reset_coords(drop=True)
                                 for ds in dsets_aligned.values()],
                                dim=source_da)

final_ds_eoc = xr.concat([ds['ssp370'].reset_coords(drop=True)
                             for ds in dsets_aligned.values()],
                            dim=source_da)
final_ds_eoc

<xarray.Dataset>
Dimensions:    (source_id: 18, year: 30, dayofyear: 365, lat: 61, lon: 121,
                bnds: 2)
Coordinates:
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * bnds       (bnds) float64 1.0 2.0
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
Data variables:
    tasmax     (source_id, year, dayofyear, lat, lon) float32 258.6 ... 255.4
Attributes:
    regrid_method:  nearest_s2d
    experiment_id:  ssp370
    source_id:      ACCESS-CM2

In [28]:
final_ds_eoc = final_ds_eoc.tasmax
final_ds_eoc

<xarray.DataArray 'tasmax' (source_id: 18, year: 30, dayofyear: 365, lat: 61,
                            lon: 121)>
array([[[[[258.5735 , 258.5735 , 258.5735 , ..., 258.5735 , 258.5735 ,
           258.5735 ],
          [260.16724, 260.07397, 259.98657, ..., 260.34375, 260.40137,
           260.16724],
          [263.55933, 263.22754, 262.91333, ..., 264.0559 , 263.7229 ,
           263.55933],
          ...,
          [262.01855, 263.08496, 263.6223 , ..., 260.19727, 261.24585,
           262.01855],
          [256.60742, 257.09595, 257.17114, ..., 255.66724, 256.34814,
           256.60742],
          [259.91162, 259.91162, 259.91162, ..., 259.91162, 259.91162,
           259.91162]],

         [[261.03174, 261.03174, 261.03174, ..., 261.03174, 261.03174,
           261.03174],
          [262.58423, 262.69946, 262.60254, ..., 262.82788, 262.80542,
           262.58423],
          [264.97314, 264.6333 , 264.2998 , ..., 265.55884, 265.17676,
           264.97314],
...
          [258.4727 , 259.8709 , 261.43506, ..., 255.4269 , 257.4986 ,
           258.4727 ],
          [257.24414, 257.66058, 258.3143 , ..., 256.13504, 256.78964,
           257.24414],
          [260.36734, 260.36734, 260.36734, ..., 260.36734, 260.36734,
           260.36734]],

         [[255.95993, 255.95993, 255.95993, ..., 255.95993, 255.95993,
           255.95993],
          [258.44458, 257.94946, 257.72528, ..., 258.51373, 258.54907,
           258.44458],
          [259.01566, 258.58304, 257.77118, ..., 260.782  , 259.4871 ,
           259.01566],
          ...,
          [267.74988, 267.0833 , 267.08966, ..., 268.49167, 268.136  ,
           267.74988],
          [264.241  , 264.11063, 263.8281 , ..., 264.457  , 264.3849 ,
           264.241  ],
          [255.44618, 255.44618, 255.44618, ..., 255.44618, 255.44618,
           255.44618]]]]], dtype=float32)
Coordinates:
  * year       (year) int64 2071 2072 2073 2074 2075 ... 2097 2098 2099 2100
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'

In [29]:
final_ds_pi = final_ds_pi.tasmax

In [33]:
%%time
# final_ds_eoc = final_ds_eoc.dropna(dim='lat',how='all')
final_ds_eoc.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').sel(source_id='AWI-CM-1-1-MR').values

CPU times: user 4.5 ms, sys: 1.57 ms, total: 6.07 ms
Wall time: 9.16 ms


array([272.5829 , 270.56686, 286.21387, 275.74948, 282.8416 , 277.98755,
       286.36908, 271.92682, 277.2629 , 279.90912, 282.954  , 268.2161 ,
       274.97894, 283.2927 , 268.0333 , 280.73755, 273.08466, 275.92004,
       279.74283, 274.79456, 291.5591 , 275.88782, 274.59494, 277.85416,
       267.45245, 265.5011 , 280.99368, 282.84323, 268.45355, 265.47726],
      dtype=float32)

In [34]:
final_ds_pi.sel(dayofyear=365).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').sel(source_id='AWI-CM-1-1-MR').values

array([272.56937, 274.75165, 272.7334 , 272.48502, 261.21567, 273.12256,
       264.64218, 263.17862, 271.2698 , 285.49225, 264.93674, 266.88162,
       266.94745, 274.6651 , 271.401  , 266.1324 , 269.2064 , 270.67603,
       265.15228, 273.17154, 273.9845 , 273.81393, 269.85202, 281.4554 ,
       277.07672, 272.94513, 266.9427 , 287.6396 , 274.03204, 275.65182],
      dtype=float32)

In [ ]:
# source_ids = list(dsets_aligned_.keys())
# source_da = xr.DataArray(source_ids, dims='source_id', name='source_id',
#                          coords={'source_id': source_ids})

# final_ds = xr.concat([ds.reset_coords(drop=True)
#                     for ds in dsets_aligned_.values()],
#                     dim=source_da)

# final_ds

In [35]:
# %%time
# final_ds_pi.to_zarr(cvals+'cmip6_pi_quantiles_annual.zarr',mode='w')
# final_ds_eoc.to_zarr(cvals+'cmip6_eoc_quantiles_annual.zarr',mode='w')

CPU times: user 17.1 s, sys: 6.25 s, total: 23.4 s
Wall time: 12.7 s


In [36]:
final_ds_pi  = xr.open_zarr(cvals+'cmip6_pi_quantiles_annual.zarr').tasmax
final_ds_eoc = xr.open_zarr(cvals+'cmip6_eoc_quantiles_annual.zarr').tasmax
final_ds_pi

<xarray.DataArray 'tasmax' (source_id: 18, year: 30, dayofyear: 365, lat: 61,
                            lon: 121)>
dask.array<open_dataset-tasmax, shape=(18, 30, 365, 61, 121), dtype=float32, chunksize=(3, 8, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * year       (year) int64 1850 1851 1852 1853 1854 ... 1876 1877 1878 1879

In [37]:
qeoc = compute_quantiles(final_ds_eoc)
qpi  = compute_quantiles(final_ds_pi)
qano = qeoc - qpi
qano

<xarray.DataArray 'tasmax' (quantile: 30, source_id: 18, dayofyear: 365,
                            lat: 61, lon: 121)>
dask.array<sub, shape=(30, 18, 365, 61, 121), dtype=float64, chunksize=(30, 3, 92, 16, 31), chunktype=numpy.ndarray>
Coordinates:
  * dayofyear  (dayofyear) int64 1 2 3 4 5 6 7 8 ... 359 360 361 362 363 364 365
  * lat        (lat) int64 -90 -87 -84 -81 -78 -75 -72 ... 72 75 78 81 84 87 90
  * lon        (lon) int64 0 3 6 9 12 15 18 21 ... 342 345 348 351 354 357 360
  * source_id  (source_id) <U17 'ACCESS-CM2' 'AWI-CM-1-1-MR' ... 'NorESM2-MM'
  * quantile   (quantile) float64 0.0 0.03448 0.06897 ... 0.931 0.9655 1.0

In [ ]:
%%time
qano_std         = qano.std(dim='source_id').dropna(dim='dayofyear')
qano_std_agmean  = global_mean(qano_std)
qano_std_agmean

In [ ]:
%%time
qano_std_agmean = qano_std_agmean.rename('tasmax_qanomaly')
qano_std_agmean.to_dataset().to_zarr(cvals+'cmip6_quantiles_ano_agmean.zarr',mode='w')

In [ ]:
qano_std_agmean = xr.open_zarr(cvals+'cmip6_quantiles_ano_agmean.zarr').tasmax_qanomaly
qano_std_agmean.values

In [ ]:
qano_std_agmean.plot()
plt.xlabel('Probability p')
plt.ylabel('Global, annual mean of standard deviation (K) ')
plt.title(r'Global, annual mean of stdev in $Q_f(p) -Q_i(p)$ among 18 CMIP6 models')
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
plt.text(0.05, 0.95, r'18 models, $3^{\circ} \times 3^{\circ}$ grid', transform=plt.gca().transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
plt.ylim(0,1.8)
plt.xlim(0,1.0)

In [21]:
#################################### Land and Ocean-only curves ########################

In [ ]:
################# Extra #################

In [ ]:
# #Iterate Over dsets_ Dictionary:
# for k, v in tqdm(dsets_.items()):
#     expt_dsets = v.values()
    
#     if any([d is None for d in expt_dsets]):
#         print(f"Missing experiment for {k}")
#         continue
    
#     dsets = [v[expt].pipe(process_data)
                             
#                       for expt in expts]
    
    
#     # Check if any of the experiment datasets is empty and skip this model if so
#     # print('k,v=',k,dsets)
#     # print('-------')
#     if any([ds is None for ds in dsets]):
#         print(f"Skipping {k} because one of the experiment datasets is empty")
#         continue
        
#     # align everything along model dim
#     dsets_aligned[k] = xr.concat(dsets, join='outer',
#                                  dim=expt_da)
#     print(dsets_aligned[k])
#     print('-----------------')

In [ ]:
# def remove_feb29(ds):
#     """
#     Remove February 29th for leap years and adjust day indices.
    
#     Parameters:
#     - ds (xarray.Dataset): Input dataset

#     Returns:
#     - xarray.Dataset: Modified dataset without February 29th for leap years.
#     """
    
#     # Check if the dataset has 366 days
#     if ds.dims['dayofyear'] == 366:
#         # Identify leap years
#         leap_years = [year for year in ds['year'].values if is_leap(year)]
        
        
#          # Create a mask to identify February 29 in leap years
#         mask_feb29 = (ds['dayofyear'] == 60) & ds['year'].isin(leap_years)
        
#         # Drop the values for February 29 in leap years
#         ds = ds.where(~mask_feb29, drop=True)
        
#         # Now, adjust the day indices for days after Feb 29 in leap years
        
#         mask_postfeb29 = (ds['dayofyear'] > 60) & ds['year'].isin(leap_years)
#         ds['dayofyear'].values[mask_postfeb29] -= 1
        
#         # if ds.dims['dayofyear'] == 366:
#         #     print(ds.sel(dayofyear=366).sel(lat=chicago_lat,lon=chicago_lon,method='nearest').values)
        
# #         # Drop dayofyear=60 only for leap years
# #         ds = ds.where(~((ds['dayofyear'] == 60) & ds['year'].isin(leap_years)), drop=True)
        
# #         # Adjust dayofyear indices for days after February 29th only for leap years
# #         dayofyear = ds['dayofyear'].values
# #         mask = (dayofyear > 60) & np.isin(ds['year'].values, leap_years)
# #         dayofyear[mask] -= 1
# #       ds = ds.assign_coords(dayofyear=('dayofyear', dayofyear))

#     return ds



In [22]:
# #Select data for specific location to plot
# df_loc = final_ds.to_dataframe().reset_index()
# df_eoc = final_ds.sel(experiment_id='ssp370').sel(year= slice(2071,2100)).to_dataframe().reset_index()
# df_pi  = final_ds.sel(experiment_id='historical').sel(year= slice(1850,1879)).to_dataframe().reset_index()
# df_loc

In [23]:
# sns.relplot(data=df_loc,x="year", y="tasmax", hue='experiment_id',
#             kind="line", errorbar="sd", aspect=2);

In [24]:
# quantiles = np.linspace(0,1.0,30)
# quantiles

In [25]:
# df_eoc_quants                  = df_eoc.groupby('source_id')['tasmax'].quantile(quantiles).reset_index()
# df_eoc_quants.columns          = ['source_id','quantiles','tasmax']
# df_eoc_quants['experiment_id'] = 'ssp370'
# df_eoc_quants

In [26]:
# df_pi_quants                  = df_pi.groupby('source_id')['tasmax'].quantile(quantiles).reset_index()
# df_pi_quants.columns          = ['source_id','quantiles','tasmax']
# df_pi_quants['experiment_id'] = 'historical'
# df_pi_quants

In [27]:
# df_quants = pd.concat([df_eoc_quants, df_pi_quants],ignore_index=True)
# df_quants 

In [28]:
# g = sns.relplot(data=df_eoc_quants,x="quantiles", y="tasmax",hue='experiment_id',
#             kind="line", errorbar="sd", aspect=2);
# g.fig.suptitle('son 30, EOC tasmax at Chicago: 21 CMIP6 models')

In [29]:
# g = sns.relplot(data=df_quants,x="quantiles", y="tasmax",hue='experiment_id',
#             kind="line", errorbar="sd", aspect=2);
# g.fig.suptitle('son 30, tasmax at Chicago: 21 CMIP6 models')

In [30]:
############## Anomalies ###########################

In [31]:
# # Pivot the table based on experiment_id
# df_pivot = df_quants.pivot_table(index=['source_id', 'quantiles'], columns='experiment_id', values='tasmax')
# df_pivot

In [32]:
# # Calculate the difference
# df_pivot['tasmax_ano'] = df_pivot['ssp370'] - df_pivot['historical']
# df_pivot
# # Reset the index to turn multi-index back to columns
# df_quant_ano = df_pivot.reset_index()[['source_id', 'quantiles', 'tasmax_ano']]
# df_quant_ano

In [33]:
# g = sns.relplot(data=df_quant_ano,x="quantiles", y="tasmax_ano",
#             kind="line", errorbar="sd", aspect=2);
# g.fig.suptitle('son 30, tasmax anomaly at Chicago: 21 CMIP6 models')

In [34]:
# # Group by quantiles and compute the standard deviation for Tasmax
# df_quantano_std = df_quant_ano.groupby('quantiles')['tasmax_ano'].std().reset_index()

# # Rename the column for clarity
# df_quantano_std.rename(columns={'tasmax_ano': 'tasmax_sdev'}, inplace=True)
# df_quantano_std

In [35]:
# g = sns.relplot(data=df_quantano_std,x="quantiles", y="tasmax_sdev",
#             kind="line", errorbar="sd", aspect=2);
# g.fig.suptitle('son 30, tasmax std at Chicago: 21 CMIP6 models')